In [1]:
import sys
sys.path.append('../unseen')
import glob

from dask.distributed import Client, LocalCluster
import xclim
import xarray as xr

import fileio

In [6]:
cluster = LocalCluster()
client = Client(cluster)

/g/data/e14/dbi599/miniconda3/envs/unseen/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 45847 instead
  warnings.warn(


In [7]:
client

Client Scheduler: tcp://127.0.0.1:33579 Dashboard: http://127.0.0.1:45847/status,Cluster Workers: 4 Cores: 8 Memory: 28.00 GiB


In [2]:
history = fileio.get_new_log()
print(history)

Tue Jul 06 13:54:07 2021: /g/data/e14/dbi599/miniconda3/envs/unseen/bin/jupyter notebook /home/599/dbi599/unseen/notebooks/preprocess.ipynb


# CAFE

In [3]:
cafe_files = glob.glob('/g/data/xv83/ds0092/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-199[0,1,2,3]??01/ZARR/atmos_isobaric_daily.zarr.zip')
cafe_files.sort()
cafe_files

['/g/data/xv83/ds0092/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-19900501/ZARR/atmos_isobaric_daily.zarr.zip',
 '/g/data/xv83/ds0092/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-19901101/ZARR/atmos_isobaric_daily.zarr.zip',
 '/g/data/xv83/ds0092/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-19910501/ZARR/atmos_isobaric_daily.zarr.zip',
 '/g/data/xv83/ds0092/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-19911101/ZARR/atmos_isobaric_daily.zarr.zip',
 '/g/data/xv83/ds0092/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-19920501/ZARR/atmos_isobaric_daily.zarr.zip',
 '/g/data/xv83/ds0092/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-19921101/ZARR/atmos_isobaric_daily.zarr.zip',
 '/g/data/xv83/ds0092/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-19930501/ZARR/atmos_isobaric_daily.zarr.zip',
 '/g/data/xv83/ds0092/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-19931101/ZARR/atmos_isobaric_daily.zarr.zip']

In [4]:
cafe_kwargs = {'metadata_file': '../config/dataset_cafe.yml',
               'no_leap_days': True,
               'region': 'TAS-POINT',
               'variables': ['pr'],
               'units' : {'pr': 'mm/day'},
               'time_freq': 'A-DEC',
               'time_agg': 'sum',   
              }

In [5]:
cafe_outfile = '/g/data/xv83/dbi599/pr_cafe-c5-d60-pX-f6_19910501-19931101_A-DEC-sum_cafe-grid-TAS-POINT.zarr.zip'

In [9]:
cafe_ds = fileio.open_mfforecast(cafe_files, **cafe_kwargs)
cafe_ds.attrs['history'] = history
cafe_ds

<xarray.Dataset>
Dimensions:    (ensemble: 96, init_date: 8, lead_time: 11)
Coordinates:
  * lead_time  (lead_time) int64 0 1 2 3 4 5 6 7 8 9 10
  * ensemble   (ensemble) int64 1 2 3 4 5 6 7 8 9 ... 88 89 90 91 92 93 94 95 96
  * init_date  (init_date) datetime64[ns] 1990-12-31 1990-12-31 ... 1993-12-31
    time       (lead_time, init_date) datetime64[ns] 1990-12-31 ... 1994-01-10
Data variables:
    pr         (init_date, lead_time, ensemble) float32 dask.array<chunksize=(1, 1, 96), meta=np.ndarray>
Attributes:
    comment:    pressure level interpolator, version 3.0, precision=double
    filename:   atmos_isobaric_daily.zarr
    grid_tile:  N/A
    grid_type:  regular
    title:      AccessOcean-AM2
    history:    Tue Jul 06 13:54:07 2021: /g/data/e14/dbi599/miniconda3/envs/...

In [10]:
cafe_ds['pr']

<xarray.DataArray 'pr' (init_date: 8, lead_time: 11, ensemble: 96)>
dask.array<concatenate, shape=(8, 11, 96), dtype=float32, chunksize=(1, 1, 96), chunktype=numpy.ndarray>
Coordinates:
  * lead_time  (lead_time) int64 0 1 2 3 4 5 6 7 8 9 10
  * ensemble   (ensemble) int64 1 2 3 4 5 6 7 8 9 ... 88 89 90 91 92 93 94 95 96
  * init_date  (init_date) datetime64[ns] 1990-12-31 1990-12-31 ... 1993-12-31
    time       (lead_time, init_date) datetime64[ns] 1990-12-31 ... 1994-01-10
Attributes:
    cell_methods:   time: mean
    interp_method:  conserve_order1
    long_name:      Total precipitation rate
    time_avg_info:  average_T1,average_T2,average_DT
    units:          mm d-1

In [11]:
cafe_ds = cafe_ds.chunk({'lead_time': 50})

In [12]:
cafe_ds['pr']

<xarray.DataArray 'pr' (init_date: 8, lead_time: 11, ensemble: 96)>
dask.array<rechunk-merge, shape=(8, 11, 96), dtype=float32, chunksize=(1, 11, 96), chunktype=numpy.ndarray>
Coordinates:
  * lead_time  (lead_time) int64 0 1 2 3 4 5 6 7 8 9 10
  * ensemble   (ensemble) int64 1 2 3 4 5 6 7 8 9 ... 88 89 90 91 92 93 94 95 96
  * init_date  (init_date) datetime64[ns] 1990-12-31 1990-12-31 ... 1993-12-31
    time       (lead_time, init_date) datetime64[ns] dask.array<chunksize=(11, 8), meta=np.ndarray>
Attributes:
    cell_methods:   time: mean
    interp_method:  conserve_order1
    long_name:      Total precipitation rate
    time_avg_info:  average_T1,average_T2,average_DT
    units:          mm d-1

In [13]:
fileio.to_zarr(cafe_ds, cafe_outfile)

# AWAP

In [14]:
awap_infile = '/g/data/xv83/ds0092/data/csiro-dcfp-csiro-awap/rain_day_19000101-20201202_cafe-grid.zarr/'
awap_outfile = '/g/data/xv83/dbi599/pr_awap_cafe-grid-TAS-POINT.zarr.zip'

In [15]:
awap_kwargs = {'metadata_file': '../config/dataset_awap.yml',
               'no_leap_days': True,
               'region': 'TAS-POINT',
               'variables': ['pr'],
               'units' : {'pr': 'mm/day'},
               'time_freq': 'A-DEC',
               'time_agg': 'sum',  
              }

In [16]:
awap_ds = fileio.open_file(awap_infile, **awap_kwargs)
awap_ds.attrs['history'] = history
awap_ds

<xarray.Dataset>
Dimensions:  (time: 121)
Coordinates:
  * time     (time) object 1900-12-31 00:00:00 ... 2020-12-31 00:00:00
Data variables:
    pr       (time) float64 dask.array<chunksize=(1,), meta=np.ndarray>
Attributes:
    history:  Tue Jul 06 13:54:07 2021: /g/data/e14/dbi599/miniconda3/envs/un...

In [17]:
awap_ds['pr']

<xarray.DataArray 'pr' (time: 121)>
dask.array<concatenate, shape=(121,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 1900-12-31 00:00:00 ... 2020-12-31 00:00:00
Attributes:
    analysis_time:                 2010-10-23 14:49:46 UTC
    analysis_version_number:       3.01
    cell_methods:                  time: sum
    frequency:                     daily
    grid_mapping:                  crs
    length_scale_for_analysis:     80.0
    long_name:                     Daily precipitation
    number_of_stations_reporting:  2757
    source:                        Interpolated surface observations
    standard_name:                 lwe_thickness_of_precipitation_amount
    units:                         mm d-1
    valid_range:                   [-1.0, 100000.0]

In [18]:
fileio.to_zarr(awap_ds, awap_outfile)